In [1]:
%%bash
pip install pandas-datareader
pip install get-all-tickers
pip install yfinance

  Created wheel for get-all-tickers: filename=get_all_tickers-1.7-cp36-none-any.whl size=4242 sha256=e6d9ec757850b96a503fc0da18f4fa8369cc61425ae15cf6ba78c5be17df86da
  Stored in directory: /root/.cache/pip/wheels/d0/8b/ef/6cd3580f4b479aef881a32bd937c282982e3d186b617a663ac
Successfully built get-all-tickers
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=a0d7cd07d35bc3a86679e635898596cb7d8db505ad23790e9bbbfa35e96e78ca
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [28]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import History
from collections import Iterable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

import time
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame
from get_all_tickers import get_tickers as gt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.spatial import distance_matrix
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import cross_val_score, ShuffleSplit
from scipy.stats.mstats import winsorize
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout

#getting the labels
news_data = pd.read_csv('Combined_News_DJIA.csv', index_col=0)
labels = news_data[['Label']]

# USER INPUT REQUIRED FOR STOCK SELECTION, DATE & DECIDING TOP MOVER THRESHOLD PERCENTAGE

In [29]:
stocks = input("Which stocks do you want to check: ")
#ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL   <-------using this as first input for test purposes(any ticker stoc symbols could be used)
date_check = input("Please enter date between 2008/08/08 and 2016/07/01 in yyyy-mm-dd format to use Market Sentiment:")
#2016-06-30        <----------- use this as second input
defined_topmover_threshold = input("Please enter % threshold to define top movers stocks:")  ## Set this to define top mover threshold percentage

Which stocks do you want to check: SOHU,NKTR
Please enter date between 2008/08/08 and 2016/07/01 in yyyy-mm-dd format to use Market Sentiment:2016-06-30
Please enter % threshold to define top movers stocks:20


In [30]:
def Convert(string): 
    li = list(string.split(",")) 
    return li
stocks=Convert(stocks) 

In [31]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd

# Tickers list
# We can add and delete any ticker from the list to get desired ticker live data
ticker_list=stocks
# We can get data by our choice by giving days bracket
start_date= '2008-08-08'
end_date=date_check
files=[]
# Create a data folder in your current dir.
def SaveData(df, filename):
  df.to_csv('./'+filename+'.csv')
def getData(ticker):
  print (ticker)
  data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
  dataname = ticker
  files.append(dataname)
  SaveData(data, dataname)

#This loop will iterate over ticker list, will pass one ticker to get data, and save that data as file.
for tik in ticker_list:
  getData(tik)

SOHU
[*********************100%***********************]  1 of 1 completed
NKTR
[*********************100%***********************]  1 of 1 completed


In [32]:
class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [33]:
def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)

# LSTM PREDICTIONS WITHOUT SENTIMENT SCORE INFORMATION

In [ ]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL

for tik in ticker_list:
  tik_data=pd.read_csv(tik+'.csv',index_col=0)
  temp_data=tik_data.drop(['Close'],axis=1)
  
  with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)
  #Scaling the variables(x) and variable(y)
  sca = temp_data.iloc[:,:]
  #Standardization of data is required as the dataset consists of variables of different scales
  sc = StandardScaler()
  ysc = StandardScaler()
  scaled_wo_sentiment = sc.fit_transform(sca)
  scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])
  
  #Y scaling for later inverse transforming predictions
  yscalesenti = with_sentiment_data
  y_with_sentiscal = ysc.fit_transform(yscalesenti.iloc[:,4].to_numpy().reshape(-1,1))
  with_sentiment_data = sc.fit_transform(with_sentiment_data)

  with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])
  x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
  y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]
  
  x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
  y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
  #Splitting data into training(70% data) and testing data(30% data)
  #x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)
  
  x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
  x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)
  TIME_STEPS = 1

  #creating lags for with sentiment scores
  
  x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
  x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)
  
  #creating lags for without sentiment scores
  
  x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
  x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)
  
  x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
  x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))
  
  x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
  x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))
  
  np.random.seed = 1   
  tf.random.set_seed = 2  
  time3=fun_t()
  history=History()
  lstm_model = Sequential()
  lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
  lstm_model.add(keras.layers.Dense(units = 1))
  lstm_model.compile(loss='mean_squared_error', optimizer='adam')
  history = lstm_model.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)
  time_lstm = list(time3.times)
  train_lstm = history.history.get('loss')
  val_lstm = history.history.get('val_loss')
  mse=val_lstm[-1]
  print("##############################################################################")
  print('MSE for',tik,'stock excluding Sentiment scores information:',mse)
  predicted_stock_price=lstm_model.predict(x_test_without_sentiment_scores)
  predicted_stock_price = ysc.inverse_transform(predicted_stock_price)

  print("Predicted stock price inverse transformed:",predicted_stock_price[-1])

  #The below data's stock price needs to be compared with prediction for the last row from above print
  print("Previous day data:",tik_data.iloc[-2,:])
  print("Check Adj Close for comparison with the inverse transformed predicted stock price")
  #print("Actual stock price:",tik_data.iloc[-1,:])
  
  
  #Calculating the top mover category using above predictions
  percent_change = ((predicted_stock_price[-1]-tik_data.iloc[-2,4])/tik_data.iloc[-2,4])*100
  print("Percentage change expected next day",percent_change)
  if percent_change > int(defined_topmover_threshold) and percent_change > 0: 
    print(tik,"stock is predicted to be a top mover the next day")
  else: print(tik,"stock might not be a top mover the next day")
  print("##############################################################################")

##############################################################################
MSE for ZION stock excluding Sentiment scores information: 0.5847963690757751
Predicted stock price inverse transformed: [16.983372]
Previous day data: Open         2.412000e+01
High         2.450000e+01
Low          2.386000e+01
Close        2.439000e+01
Adj Close    2.208469e+01
Volume       4.786600e+06
Name: 2016-06-28, dtype: float64
Check Adj Close for comparison with the inverse transformed predicted stock price
Percentage change expected next day [-23.098904]
ZION stock might not be a top mover the next day
##############################################################################
##############################################################################
MSE for KGC stock excluding Sentiment scores information: 0.7469857931137085
Predicted stock price inverse transformed: [13.109369]
Previous day data: Open         5.020000e+00
High         5.110000e+00
Low          4.950000e+00
Close        

# LSTM PREDICTIONS USING SENTIMENT SCORE INFORMATION

In [34]:
def create_model(units,optimizer,dropout_rate):
# create model
    model = Sequential()
    model.add(keras.layers.LSTM(units = units, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='relu'))
    # Compile model
    #optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model

In [35]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL

for tik in ticker_list:
  tik_data=pd.read_csv(tik+'.csv',index_col=0)
  temp_data=tik_data.drop(['Close'],axis=1)
  
  with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)
  #Scaling the variables(x) and variable(y)
  sca = temp_data.iloc[:,:]
  #Standardization of data is required as the dataset consists of variables of different scales
  sc = StandardScaler()
  ysc = StandardScaler()
  scaled_wo_sentiment = sc.fit_transform(sca)
  scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])
  
  #Y scaling for later inverse transforming predictions
  yscalesenti = with_sentiment_data
  y_with_sentiscal = ysc.fit_transform(yscalesenti.iloc[:,4].to_numpy().reshape(-1,1))
  with_sentiment_data = sc.fit_transform(with_sentiment_data)

  with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])
  x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
  y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]
  
  x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
  y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
  #Splitting data into training(70% data) and testing data(30% data)
  #x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)
  
  x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
  x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)
  TIME_STEPS = 1

  #creating lags for with sentiment scores
  
  x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
  x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)
  
  #creating lags for without sentiment scores
  
  x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
  x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)
  
  x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
  x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))
  
  x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
  x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))
  
  np.random.seed = 1   
  tf.random.set_seed = 2  
  time3=fun_t()
  
  ########################HYPERPARAMETER TUNING################################################################################
  lstm_model = KerasClassifier(build_fn=create_model, verbose=0)
  param_grid = dict(units = [1,2],optimizer=['SGD', 'Adam'],dropout_rate=[0.1,0.2],batch_size = [7,28],epochs = [50,100,150])
  grid = GridSearchCV(estimator=lstm_model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
  grid_result = grid.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores)

  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

  params=grid_result.best_params_
  
  ##################################Tuned Paramters############################################################################
  dropout_rate=params.get('dropout_rate')
  batch_size=params.get('batch_size')
  epoch=params.get('epochs')
  opti = params.get('optimizer')
  lstm_units=params.get('units')
  
  ######################################FINAL MODEL FOR MSE CALCULATION###########################################################################
  history=History()
  lstm_model = Sequential()
  lstm_model.add(keras.layers.LSTM(units = lstm_units, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
  lstm_model.add(Dropout(dropout_rate))
  lstm_model.add(keras.layers.Dense(units = 1,activation='relu'))
  lstm_model.compile(loss='mean_squared_error', optimizer=opti)
  history = lstm_model.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs = epoch, shuffle = False, batch_size = batch_size, callbacks=[history,time3],validation_split = 0.3,verbose = 0)
  time_lstm = list(time3.times)
  train_lstm = history.history.get('loss')
  val_lstm = history.history.get('val_loss')
  mse=val_lstm[-1]

  print("##############################################################################")
  print('MSE for',tik,'stock using Sentiment scores:',mse)

  predicted_stock_price=lstm_model.predict(x_test_with_sentiment_scores)
  predicted_stock_price = ysc.inverse_transform(predicted_stock_price)
  
  print("Predicted stock price inverse transformed:",predicted_stock_price[-1])
  
  #The below data's stock price needs to be compared with prediction for the last row from above print
  print("Previous day data:",tik_data.iloc[-2,:])
  #print("Actual stock price:",tik_data.iloc[-1,:])
  print("Check Adj Close for comparison with the inverse transformed predicted stock price")
  #Calculating the top mover category using above predictions
  percent_change = ((predicted_stock_price[-1]-tik_data.iloc[-2,4])/tik_data.iloc[-2,4])*100
  print("Percentage change expected next day",percent_change)
  if percent_change > int(defined_topmover_threshold) and percent_change > 0: 
    print(tik,"stock is predicted to be a top mover the next day going above defined threshold",defined_topmover_threshold,"%")
  else: print(tik,"stock might not be a top mover the next day")
  print("##############################################################################")
  if percent_change < abs(int(defined_topmover_threshold)) and percent_change < 0: 
    print(tik,"stock is predicted to perform worse the next day and could go below the threshold -ve ",defined_topmover_threshold,"%")

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.1min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 19.6min finished


Best: 0.000720 using {'batch_size': 7, 'dropout_rate': 0.1, 'epochs': 50, 'optimizer': 'SGD', 'units': 1}
##############################################################################
MSE for SOHU stock using Sentiment scores: 0.8923661708831787
Predicted stock price inverse transformed: [56.52957]
Previous day data: Open             36.430000
High             36.980000
Low              36.130001
Close            36.590000
Adj Close        36.590000
Volume       149100.000000
Name: 2016-06-28, dtype: float64
Check Adj Close for comparison with the inverse transformed predicted stock price
Percentage change expected next day [54.49459]
SOHU stock is predicted to be a top mover the next day going above defined threshold 20 %
##############################################################################
Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 19.6min finished


Best: 0.001440 using {'batch_size': 7, 'dropout_rate': 0.1, 'epochs': 50, 'optimizer': 'SGD', 'units': 1}
##############################################################################
MSE for NKTR stock using Sentiment scores: 1.4331543445587158
Predicted stock price inverse transformed: [10.216975]
Previous day data: Open              13.58
High              13.93
Low               13.54
Close             13.76
Adj Close         13.76
Volume       1069700.00
Name: 2016-06-28, dtype: float64
Check Adj Close for comparison with the inverse transformed predicted stock price
Percentage change expected next day [-25.748726]
NKTR stock might not be a top mover the next day
##############################################################################
NKTR stock is predicted to perform worse the next day and could go below the threshold -ve  20 %
